# Trabajo MC
## Redundancy Allocation Problem with Tabu Search

#### TO-DO LIST
- Implementar búsqueda tabú x
 - <span style="color:gray">(Tener en cuenta el tamaño máximo de la lista tabú {1}) </span>
- Estudiar heurística explicada en el paper
- Aplicar la heurística al algoritmo
- Desarrollar funciones de generación de problemas
- Desarrollar funciones de representación gráfica


> {1} Según el artículo, este tamaño máximo sera dinámico (hay que investigar más esto)


---
#### Dudas

- Paso 2 del algoritmo (¿cómo se hace el incremento aleatorio de la dirección?)
- mngli vs mnli
- U-function <span style="color:red;font-size:20px"> <-- ¡¡IMPORTANTE!! </span>


---

#### Comentarios

Un multi-state series-parallel system (MSS) es un sistema que tiene un número de componentes puestos en serie. Cada componente puede tener uno o más elementos del mismo tipo puestos en paralelo para aumentar la redundancia, y por tanto la fiabilidad del sistema, pero aumentando también su coste. El objetivo es minimizar el coste del diseño del sistema cumpliendo un requisito mínimo de fiabilidad.

Una vez seleccionada una version de un elemento para un componente, sólo se puede generar redundancia con elementos de esa versión.

- Solución inicial: Cada componente del sistema contiene sólo UN elemento de la VERSIÓN 1
- Incrementar la dirección de la solución actual aleatoriamente
- Aplicar búsqueda tabú al subespacio comprendido por los vecinos de la solución actual


La disponibilidad del sistema es el score que se va a considerar para evaluar la bondad de una solución, y es todo el rollo ese de las transformadas y nosecuanto que se aplica a las fiabilidades nominales (W) de cada elemento individual: Para cada componente, los rendimientos de los distintos elementos en paralelo se suman a través de la u-transformada esa rara. Para todos los componentes del sistema, el rendimiento es el mínimo de los rendimientos de los componentes, pero también a través de la u-transformada. (Miratelo Alvaro que por más que rasco no lo entiendo)

El espacio de estados -denotado como S- tiene (N - 2s + 1) posibles particiones (desde la mínima dirección posible, 2s; hasta la máxima, N).

A la hora de aplicar TS a un subespacio dado, es necesario extraer una estructura de **vecindario** para el mismo. Ésta estructura se obtiene a partir de todos los posibles movimientos únicos aplicados a la solución actual Y. Es necesario puntualizar que durante esta operación, la dirección (suma de las versiones de cada componente y redundancia de cada uno) de los vecinos se mantiene igual respecto a la de la solución original.

Un **movimiento** consiste en sumar y restar uno al número de versión y al de redundancia de cada componente de la solución, lo cual puede realizarse de tres maneras:

- Cambiando el número de redundancia de los componentes
- Cambiando el número de versión de los componentes
- Cambiando tanto el número de redundancia como el número de versión de los componentes

De entre todos los vecinos será necesario **descartar** los que sean tabú en el momento determinado y escoger aquel considerado como **mejor solución (Y’)**. Ésta solución deberá añadirse a la **lista tabú** (una lista limitada donde se almacenan las soluciones escogidas más recientes). <span style='color:#f1c232;'>El tamaño de ésta lista tabú tenemos que ver cómo lo calculamos.</span>  
Además, permitiremos que se pueda seleccionar una solución con una disponibilidad menor a la mínima requerida (A<sub>0</sub>), de tal manera que podamos atravesar la región no factible de soluciones. Para ello **añadiremos una penalización ponderada** a la función de coste para aquellas soluciones que violen las restricciones de disponibilidad.

Criterio de parada:

In [15]:
import random

In [54]:
# Esquema general de la búsqueda tabú
# https://en.wikipedia.org/wiki/Tabu_search
class TabuSearch():
    """
    s: número de componentes del sistema
    X_max: vector indicando el máximo Xi permitido (lista)
    J_max: vector indicando el máximo Ji permitido (lista)
    A: vector de disponibilidad de cada versión de los componente (lista de listas)
    C: vector de costes de cada versión de los componentes (lista de listas)
    W: vector de rendimientos nominales de cada versión de los componentes (lista de listas)
    discount_func: función de descuentos (función)
    W_T: [(w_k,t_k),...] rendimiento w_k requerido durante t_k unidades de tiempo (lista de tuplas)
    q: penalización
    mnli: número máximo de iteraciones locales después de encontrar la mejor solución 
    mngi: número máximo de iteraciones globales después de encontrar la mejor solución
    """
    def __init__(self, s, X_max, J_max, A, C, W, discount_func, W_T, q, mnli, mngi):
        self.s = s
        self.X_max = X_max
        self.J_max = J_max
        self.A = A
        self.C = C
        self.W = W
        self.discount_func = discount_func
        self.W_T = W_T
        self.q = q
        self.mnli = mnli
        self.mngi = mngi
    
    def ts(self):
        stop_cond = self.mngi
        s0 = self.generate_s0()
        s_best = s0
        while stop_cond:
            stop_cond-=1
            s_cand = self.random_modification(s_best)
            s_sub_cand = self.ts_sub(s_cand)
            if(self.score(s_sub_cand)>self.score(s_best)):
                s_best=s_sub_cand
                stop_cond = self.mngi 
            
        return s_best
    
    def ts_sub(self, s0):
        s_best = s0
        stop_cond = self.mnli
        tabu_list = []
        tabu_list.append(s0)
        while stop_cond:
            stop_cond-=1  
            s_neighbors = self.neighborhood(s0, tabu_list)
            if(len(s_neighbors) == 0):
                continue
            s_cand = s_neighbors.pop()
            for neighbor in s_neighbors:
                if (not neighbor in tabu_list and self.score(neighbor)>self.score(s_cand)):
                    s_cand = neighbor
            if(self.score(s_cand)>self.score(s_best)):
                s_best=s_cand
                stop_cond = self.mnli
            tabu_list.append(s_cand)
            if(len(tabu_list) > self.mnli ):
                del tabu_list[0]
                          
        return s_best
    
    
    def neighborhood(self, solution, tabu_list):
        # Devuelve la lista de vecinos de solution
        def is_valid_neighbor(n, i1, i2):
            ''' Checkea si la modificación de Xi y/o de Ji de un vecino dado es válida (si estos valores se sitúan 
            entre 1 y X_max/J_max) y comprueba si no se encuentra ya en la lista tabu'''
            return (n[i1] in range(1, (self.X_max + self.J_max)[i1]+1) and
               n[i2] in range(1, (self.X_max + self.J_max)[i2]+1) and n not in tabu_list)

        def add_subtract_one_at_indexes(l,i1,i2):
            ''' Dados dos índices (i1,i2), se aplican las modificaciones (+1,-1) y (-1,+1) en los elementos
             correspondientes a las posiciones de dichos índices en la solución dada. Además, se checkea si
             dicha modificación es válida'''
            n1,n2 = l.copy(), l.copy()        
            n1[i1],n1[i2] = n1[i1]+1,n1[i2]-1
            n2[i1],n2[i2] = n2[i1]-1,n2[i2]+1            
            return [ n for n in [n1, n2] if is_valid_neighbor(n, i1, i2)]

        return [ n for i1 in range(len(solution)-1) 
                     for i2 in range(i1+1, len(solution)) 
                        for n in add_subtract_one_at_indexes(solution,i1,i2)]
    
    
    def score(self, solution):
        # Devuelve el score de solution de acuerdo con la heuristica del articulo
        return 1
    
    def generate_s0(self):
        s0 = [1,1]*self.s
        return s0
    
    def address(self, solution):
        return sum(solution)    
    
    def random_modification(self, solution):
        return solution


In [57]:
#%timeit -r 3 ns = neighborhood([ random.choice(range(10)) for _ in range(500)], [10]*500, [10]*500)

In [58]:
t = TabuSearch(4, [10]*4, [10]*4, [],
           [], [], (lambda x: x), [], 1, 20, 50)

In [59]:
t.ts_sub([2,4,5,4,8,4,2,4])

[2, 4, 5, 4, 8, 4, 2, 4]

In [60]:
t.ts()

[1, 1, 1, 1, 1, 1, 1, 1]